In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


from mnist import MNIST

mndata = MNIST('samples')

x_train, y_train = mndata.load_training()

x_test, y_test = mndata.load_testing()

In [2]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [3]:
y_train = pd.get_dummies(y_train)
y_train = np.array(y_train)

y_test = pd.get_dummies(y_test)
y_test = np.array(y_test)

In [4]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

Functions and etc.

In [5]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_derv(z):
    return z * (1 - z)

def softmax(z):
    exps = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(pred, real):
    n = real.shape[0]
    difference = pred - real
    return difference/n

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss

def tanh(z):
    t=(np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
    dt=1-t**2
    return t

def tanh_derv(z):
    dt = 1-z**2
    return dt
    #call by using tanh(z)[0] and tanh(z)[1]
    
def relu(z):
    return np.maximum(0,z)


In [6]:
class neuralnet():
    def __init__(self,x_train,y_train):
        self.x_train = x_train
        hiddenlayernodes = 130
        self.y_train = y_train
        
        self.learning_rate = 5
        self.epoch_no = 1
        attributes = 784
        labels = 10
        
        
        self.w4 = np.random.randn(4,5)
        self.w1 = np.random.randn(attributes,hiddenlayernodes)
        self.b1 = np.zeros((1,hiddenlayernodes))
        
        self.w2 = np.random.randn(hiddenlayernodes,hiddenlayernodes)
        self.b2 = np.zeros((1,hiddenlayernodes))
        
        self.w3 = np.random.randn(hiddenlayernodes,labels)
        self.b3 = np.zeros((1,labels))
        
        
    def feedforward(self):
        z1 = np.dot(self.x_train,self.w1) + self.b1
        self.layer1 = sigmoid(z1)
        
        z2 = np.dot(self.layer1,self.w2) + self.b2
        self.layer2 = sigmoid(z2)
        
        z3 = np.dot(self.layer2,self.w3) + self.b3
        self.layer3 = softmax(z3)

        
    def backpropogation(self):
        loss = error(self.layer3,self.y_train)
        
#         if self.epoch_no % 5 == 0:
        #print("The Loss after epoch ",self.epoch_no," is: ",loss)
        print(loss)
        self.epoch_no += 1
        
        #no derivative for z3 as 
        layer3dt = cross_entropy(self.layer3,self.y_train)
        
        z2dt = np.dot(layer3dt,self.w3.T)
        layer2dt = z2dt * sigmoid_derv(self.layer2)
        
        z1dt = np.dot(layer2dt,self.w2.T)
        layer1dt = z1dt * sigmoid_derv(self.layer1)
        
        
        self.w3 -= self.learning_rate * np.dot(self.layer2.T,layer3dt)
        self.b3 -= self.learning_rate * np.sum(layer3dt , axis = 0, keepdims = True)
        
        self.w2 -= self.learning_rate * np.dot(self.layer1.T,layer2dt)
        self.b2 -= self.learning_rate * np.sum(layer2dt , axis = 0 , keepdims = True)
        
        self.w1 -= self.learning_rate * np.dot(self.x_train.T,layer1dt)
        self.b1 -= self.learning_rate * np.sum(layer1dt, axis = 0 , keepdims = True)
        
    def epochoutput(self,data):
        self.x_train = data
        self.feedforward()
        return self.layer3.argmax()
        

fullyconnectedlayer = neuralnet(x_train/255, y_train)

for i in range(200):
    fullyconnectedlayer.feedforward()
    fullyconnectedlayer.backpropogation()
    
print("DOne")
def get_acc(x, y):
    acc = 0
    for xx,yy in zip(x, y):
        s = fullyconnectedlayer.epochoutput(xx)
        if s == np.argmax(yy):
            acc +=1
    return acc/len(x)*100
	
print("Training accuracy : ", get_acc(x_train/255, y_train))
print("Test accuracy : ", get_acc(x_val/255, y_val))


14.78574145305265
28.214533707264525
35.75528844413107
42.435846274923314
43.18120494900118
32.34314556624406
29.11459610465705
23.682947258093726
21.983843169069324
20.584458813799234
19.501636139069504
18.258083454193265
20.829198530619113
19.568239505656003
19.19858457170892
15.479150362689747
13.607860346936159
12.68442925154032
11.826031953339259
12.80693169949603
12.150204200940102
11.670279353618868
11.317576918240187
11.199590974697125
11.807303123201102
10.60935248799811
9.633216700478322
8.379203648710973
7.617863125845719
7.331179862218673
6.189289326271164
5.646242159476529
5.398188111613328
4.960499955532656
2.827343321738811
2.768971520743941
2.2446927143933144
1.8876120827477987
2.1293873831701506
1.9990557204017931
2.0478924840507813
1.8594074324020853
2.1873167756426
1.886429530297893
2.009530085849439
1.938283926836204
1.901147914696914
1.9709011971461217
1.6829311770320772
1.992603145400295
2.217626646810904
2.0615241368932264
1.923889207781149
2.014665071637227
1.84